In [2]:
# ipynb 파일이 분리되어 있는 관계로 이전 파일의 실행 내역 복사

import pandas as pd
data = pd.read_csv("turnover.csv")

data.salary = data.salary.astype('category')
data.salary = data.salary.cat.reorder_categories(['low', 'medium', 'high'])
data.salary = data.salary.cat.codes

departments = pd.get_dummies(data.department)
departments = departments.drop("technical", axis=1)
data = data.drop("department", axis=1)
data = data.join(departments)

target = data.left
features = data.drop("left", axis=1)
from sklearn.model_selection import train_test_split
target_train, target_test, features_train, features_test = train_test_split(target, features, test_size=0.25)

from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(random_state=42)
model.fit(features_train, target_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

## Evaluating the turnover prediction model

### 퇴직율 classifier 조정하기
좀 더 나은 결과를 얻기 위해 overfitting 제거 작업 수행하는데, 위 결과를 보면 아래와 같은 overfitting이 발생
- training 데이터의 정확도 : 100%
- Testing 데이터의 정확도 : 97.65%

그렇다면 overfitting을 줄이거나 없애기 위해 무엇을 조정해야 할까?
- tree의 최대 단계 수를 제한(예, 5단계까지만 tree 생성)
- 개별 단계(leaf 또는 node)의 크기를 제한(예, 노드별로 최대 100명까지만 허용)

#### 먼저 단계 수(depth) 제한

In [2]:
# tree의 단계 수르 5개로 제한
model_depth_5 = DecisionTreeClassifier(max_depth=5, random_state=42)

# 모델 fitting
model_depth_5.fit(features_train, target_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

In [3]:
# 단계 수를 제한 한 다음에 train 데이터 대한 예측결과의 정확도를 점수로 환산해보면..
model_depth_5.score(features_train, target_train)*100

97.537558894123919

In [4]:
# 테스트 데이터 정확도를 점수로 확인
model_depth_5.score(features_test, target_test)*100

97.493333333333325

#### 다음으로 노드 크기(인원) 제한

In [5]:
# 노드의 크기를 200명으로 제한
model_sample_200 = DecisionTreeClassifier(min_samples_leaf=200, random_state=42)

# 모델 fitting
model_sample_200.fit(features_train, target_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=200, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

In [6]:
# 노드 크기를 제한 한 다음에 train 데이터와 test 데이터에 대한 예측결과의 정확성을 점수로 환산해보면..
model_sample_200.score(features_train, target_train)*100

95.466263667881591

In [7]:
# 테스트 데이터 정확성 확인
model_sample_200.score(features_test, target_test)*100

95.439999999999998

위와 같은 방식으로 하면 각각의 정확도는 떨어지지만 두 데이터셋 간의 차이(거리)는 줄어든다.</p>
즉, overfitting이 줄어들면서 더 현실적인 예측모델이 된다.

(Depth = 5와 sample =200을 합치면 sample=200의 결과와 같다.)

지금까지는 정확도(accuracy)만을 계산했다. 그러나 정확도 지표 하나만으로는 최적의 모델을 찾는데 역부족.</p>
왜냐고?
일반적인 Accuracy 점수는 분리된 데이터군(class, 여기서는 재직자와 퇴직자군)에 관한 정보를 보여줄수 없기 때문.</br>
예) 재직자의 76% = "모두가 남아있을"거라는 예측의 정확도가 76%의 확률

그래서 Confustion matrix를 사용해서 예측오류(prediction error)를 줄이는 방법을 찾아내야 한다.
- 퇴직자에 대한 정확도 평가 --> Recall 점수 산정 : 퇴직자인데 퇴직자 아니라고 예측한 비율(FN)
- 재작자에 대한 정확도 평가 --> Specificity 점수 산정 : 퇴직자 아닌데 퇴직자라고 예측한 비율(FP)

### 정밀도(precision)의 accuracy metircs 계산하기

정밀도 : 퇴직할 것이라고 예측한 사람들 중 실제로 퇴직한 사람들의 비율</p>
Precision score는 classification 알고리즘의 정확도를 재는 중요한 측정지표(metrics)로 True Positive를 True Positive와 False Positive의 합으로 나눈 것.

 = # of True Positives / # of True Positivies + # of False Positives

- True Positives = 실제 퇴직한 직원을 정확하게 예측한 경우
- False Positives = 퇴직을 안했는데 퇴직할 것이라고 잘못 예측한 경우

- 정밀도(precision score)가 가장 높은 것은 1, 가장 낮은 것은 0

In [11]:
# precision score를 계산할 함수 호출
from sklearn.metrics import precision_score

# features 데이터를 이용해서 어떤 직원이 이직할 것인지를 예측
prediction = model.predict(features_test)

# target 데이터에 대한 예측비교를 통해 precision score 계산
precision_score(target_test, prediction)

0.94481236203090513

### 재현율(Recall)의 accuracy metircs 계산하기

재현율 : 퇴직할 것이라고 예측했는데 실제로 그 예측이 맞을 확률</p>
Recall score는 classification 알고리즘의 정확도를 재는 중요한 측정지표(metrics)로 일명 민감도(Sensitivity)라고도 함

 = # of True Positives / # of True Positivies + # of False Negatives

- False Negatives가 없으면 Recall score는 최대치가 됨 
- 재현율(recall score)이 가장 높은 것은 1, 가장 낮은 것은 0

In [3]:
# recall score를 계산하는 함수 호출
from sklearn.metrics import recall_score

# 퇴직자를 예측할 초기 모델 수립 : features test 데이터를 대상으로 
prediction = model.predict(features_test)

# 대상데이터로 예측한 결과와 비교하여 recall score 계산
recall_score(target_test, prediction)

0.96851248642779586

### 재직자와 퇴직자 모두를 대상으로 분석하기

- Recall 점수 -> False Negative의 수에 비중을 둠 
- Specificity 점수 -> False Positive의 수에 비중을 둠

여기서는 퇴직자를 예측하는 것이므로,</p>
Recall 점수는 퇴직자를 대상으로 하는 예측분석에, Specificity 점수는 재직자를 대상으로 하는 예측분석에 유용

그러나, 분석 대상을 하나만으로 한정하면 다른 대상에 대한 예측의 정확도(accuracy)는 떨어짐</p>
그래서 두 대상 모두에 대해 좋은 예측력을 얻기 위해서는 AUC(Area Under Curve) 점수를 사용

#### AUC 점수
수직 축 : Recall </p>
수평 축 : 1 - Specificity

In [4]:
# ROC와 AUC 점수를 계산하는 함수 호출 : roc_auc_score
from sklearn.metrics import roc_auc_score

# 퇴직자 예측을 위한 모델 수립
prediction = model.predict(features_test)

# 예측값과 대상데이터와의 비교를 통해 ROC와 AUC 점수 계산
roc_auc_score(target_test, prediction)

0.97453549383249105

### Class Imbalance
분석대상 대조군(class)간의 데이터 차이(imbalance). <b>지금까지의 분석은 이 class imbalance 상태에서 이루어진 것임</b> </p>
recall과 precision 점수의 차이에서 봤듯이, class imbalance는 예측결과에 심각한 영향을 미침</p>

class imbalance 문제 해결 방법
- 각각의 class에 다른 가중치를 부여하여 균형을 맞춤(=balanced) : class_weight
- over-sampling
- under-sampling
- Synthetic Minority Oversampling Technique(SMOTE)

#### Balancing classes
Decision tree에서 class imbalance를 해소하기 위해서는 
- 각각의 class(분석대상 군)에 가중치를 부여 : class_weight='balanced'

In [5]:
# DecisionTreeClassifier 초기화
model_depth_5b = DecisionTreeClassifier(max_depth=5, class_weight='balanced', random_state=42)

# 모델 fitting
model_depth_5b.fit(features_train, target_train)

# 테스트 데이터를 대상으로 예측점수 산정
model_depth_5b.score(features_test, target_test)*100

93.733333333333334

### Employee attrition 모델의 비교

Decision tree에서 불필요한 tree의 가지(=단계)를 줄이는 pruned tree를 사용하여 'balanced model'과 'imbalanced model'을 비교 
(여기서는 max_depth=7로 조정)

imbalanced model은 recall 점수와 ROC/AUC 점수를 통해 위에서 이미 만들어 봤으니 이제는 같은 방법으로 balanced model을 만들어 본다.

#### class balance model 만들기

In [13]:
# 이전에 계산했던 recall score와 ROC/AUC score 조회
print(recall_score(target_test,prediction))
print(roc_auc_score(target_test,prediction))

# class balance를 위한 비교 모델 생성 - class_weight 사용
model_depth_7b = DecisionTreeClassifier(max_depth=7, class_weight='balanced', random_state=42)

# 새로운 모델(balanced model)을 train 데이터에 fitting
model_depth_7b.fit(features_train,target_train)

# test 데이터를 대상으로 예측
prediction_b = model_depth_7b.predict(features_test)

# 새로운 recall score와 ROC/AUC score를 각각 출력
print(recall_score(target_test, prediction_b))
print(roc_auc_score(target_test, prediction_b))

0.968512486428
0.974535493832
0.940282301846
0.962364551418
